In [ ]:
%%capture
#Libraries
!pip install gradio
!pip install datasets
!pip install openai
!pip install fuzzywuzzy python-Levenshtein
!pip install scikit-learn
!pip install pyspellchecker

#Imports
import gradio as gr
from datasets import load_dataset
from openai import OpenAI
from google.colab import userdata
from fuzzywuzzy import process
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from spellchecker import SpellChecker
import json
import os

In [ ]:
#Retrieving the secret API key
apikeykey = userdata.get('OPENAI_API_KEY')
#Initializing the OpenAI client
client = OpenAI(api_key = apikeykey)
GPT_MODEL = "gpt-4"

snippets_dataset = load_dataset("P8IxD/Snippets")

#Initializing a new dataset for saved snippets
saved_snippets_dataset = []


def get_openai_response_code(prompt):
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages = [{"role": "system", "content": "You are a code generator tasked with creating a cohesive code snippet from the 'code' column in the provided dataset based on the user's input. Combine relevant snippets in JSX and Tailwind CSS to form a complete functionality. Ensure the output contains only code (no comments, symbols, or additional text) and is formatted as a single traditional code block, for example: ‘[Code]’."},
                    {"role": "user", "content": "Generate code for: " + prompt}],
        max_tokens = 200,
        temperature = 0.7
    )

    return response.choices[0].message.content


def get_openai_response_name(prompt):
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages = [{"role": "system", "content": "Create a new snippet based on the prompt given by the user utilising the existing snippets found in the provided dataset. ChatGPT’s role is to create a name for the generated snippet, used for web development. The snippet name must take inspiration from the ‘name’ column in the provided dataset and be concise with a maximum length of 3 words. Craft the name with professionalism and clarity, incorporating IT terminology where appropriate."},
                    {"role": "user", "content": "Generate name for: " + prompt}],
        max_tokens = 10,
        temperature = 0.4
    )

    return response.choices[0].message.content


def get_openai_response_description(prompt):
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages = [{"role": "system", "content": "Create a new snippet based on the prompt given by the user utilising the existing snippets found in the provided dataset. ChatGPT’s role is to create a description for the generated snippet, used for web development. The snippet description must take inspiration from the ‘description’ column in the provided dataset and be concise with a maximum length of 20 words. Craft the description with professionalism and clarity, incorporating IT terminology where appropriate."},
                    {"role": "user", "content": "Generate description for: " + prompt}],
        max_tokens = 25,
        temperature = 0.6
    )

    return response.choices[0].message.content


def correct_spelling(user_input, dataset):
  words = set()
  for entry in dataset:
    for word in entry.split():
      words.add(word.lower())
  #Checks if the original word is already present in the dataset
  if user_input.lower() in words:
    return user_input.lower()
  #Using Fuzzy matching to find most similar word in the dataset
  corrected_word, _ = process.extractOne(user_input, words)

  return corrected_word

#Functions which combines names, descriptions and codes for GPT4 processing
def generate_combined_name(names):
  combined_prompt = "Generate a name for: " + " ".join(names)
  generated_name = get_openai_response_name(combined_prompt)
  return generated_name

def generate_combined_description(descriptions):
  combined_prompt = "Generate a description for: " + " ".join(descriptions)
  generated_description = get_openai_response_description(combined_prompt)
  return generated_description

def generate_combined_code(codes):
  combined_prompt = "Generate code for: " + " ".join(codes)
  generated_code = get_openai_response_code(combined_prompt)
  return generated_code



def get_snippets_info(user_input):
    user_input = user_input.lower().split()
    #Gets a list of all snippet names and components
    snippet_info = [entry['Name'].lower() + ' ' + ' '.join(entry['Description']).lower() for entry in snippets_dataset['train']]
    #Corrects spelling of user input
    corrected_input = [correct_spelling(word, snippet_info) for word in user_input]
    vectorizer = TfidfVectorizer()
    relevance_threshold = 0.2

    combined_names = []
    combined_descriptions = []
    combined_codes = []

    for word in corrected_input:
        snippet_info.append(word)
        #Computing TF-IDF matrix
        tfidf_matrix = vectorizer.fit_transform(snippet_info)
        #Computing cosine similarity between user's input and all snippet descriptions
        cosine_similarities = cosine_similarity(tfidf_matrix[-1:], tfidf_matrix[:-1])
        #Finding indices of the relevant snippets
        relevant_indices = [i for i, score in enumerate(cosine_similarities[0]) if score > relevance_threshold]

        combined_name = ""
        combined_description = ""
        combined_code = ""

        #Iterating through the relevant indices to collect names, descriptions and codes
        for index in relevant_indices:
            name = snippets_dataset['train'][int(index)]['Name']
            description = snippets_dataset['train'][int(index)]['Description']
            code = snippets_dataset['train'][int(index)]['Code']
            #Concatenating the names, descriptions and codes
            combined_name += name + " "
            combined_description += description + " "
            combined_code += code + " "
        #Appends combined names, descriptions and codes to their corresponding lists for "AI Processing"
        combined_names.append(combined_name.strip())
        final_combined_name = generate_combined_name(combined_names)

        combined_descriptions.append(combined_description.strip())
        final_combined_description = generate_combined_description(combined_descriptions)

        combined_codes.append(combined_code.strip())
        final_combined_code = generate_combined_code(combined_codes)

    return final_combined_name, final_combined_description, final_combined_code


def save_snippet(name, description, code):

  if "Could not create a snippet" in name or "No snippet to save" in name or "Snippet saved successfully" in name:
    return "No snippet to save"

  if not name.strip() and not description.strip() and not code.strip():
    return "No snippet to save"
  #Create a dictionary for the snippet
  snippet = {
      'Name': name,
      'Descriptions': description,
      'Code': code
  }
  saved_snippets_dataset.append(snippet)
  #Checks if the file already exists
  if os.path.exists('saved_snippets.json'):
    #Load existing data if the file exists
    with open('saved_snippets.json', 'r') as f:
      data = json.load(f)

  else:
    #Initialize empty list if file doesn't exist
    data = []

  data.append(snippet)
  #Write data back to the file
  with open('saved_snippets.json', 'w') as f:
    json.dump(saved_snippets_dataset, f)

  return "Snippet saved successfully!"

#Creating the interface
with gr.Blocks() as interface:
  gr.Markdown("AI Buddy - Enter instructions for the snippet you want to generate.")
  with gr.Row():
      user_input = gr.Textbox(label="Input")

  with gr.Row():
      submit_button = gr.Button("Submit")
      save_button = gr.Button("Save")

  with gr.Row():
    with gr.Column(scale=0.5, min_width=100):
      name_output = gr.Textbox(label="Name")
      descriptions_output = gr.Textbox(label="Description")
    code_output = gr.Code(label="Code")

  submit_button.click(get_snippets_info, inputs=user_input, outputs=[name_output, descriptions_output, code_output])
  save_button.click(save_snippet, inputs=[name_output, descriptions_output, code_output], outputs=name_output)

#Launch the interface
interface.launch()

/usr/local/lib/python3.10/dist-packages/gradio/layouts/column.py:53: UserWarning: 'scale' value should be an integer. Using 0.5 will cause issues.
  warnings.warn(


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c20334775ef6cee82f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
